In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.6 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelWithLMHead

# Fixing the random seed
RANDOM_SEED = 1729
torch.manual_seed(RANDOM_SEED)

# CUDA option
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model/tokenizer name or path
model_name_or_path = 'turingmachine/hupd-t5-small'
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token
# Model
model = AutoModelWithLMHead.from_pretrained(model_name_or_path)
model.to(device)

# Load the Excel file
df = pd.read_excel('/content/drive/MyDrive/All_Patent.xlsx')
df = df.head(50)

# Extract the desired columns
columns_to_keep = ['Filename', 'Abstract', 'Claims']  # Add other column names here
df_subset = df[columns_to_keep]

# Store the 'Abstract' and 'Claims' columns in one variable
abstracts = df['Abstract']

# List to store the generated text
generated_texts = []

# Generate text for each abstract
for abstract in abstracts:
    inputs = tokenizer(abstract, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(inputs.input_ids, max_length=512, num_return_sequences=1, early_stopping=True)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_texts.append(generated_text)

# Add the generated text to the DataFrame
df_subset['Abstract_Summary'] = generated_texts

# Save the data frame to a new Excel file
output_file = '/content/drive/MyDrive/abstract_summary_code.xlsx'
df_subset.to_excel(output_file, index=False)